# Tolstoy
https://habrahabr.ru/post/342738/

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [13]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [14]:
text[:110]

'ЧАСТЬ ПЕРВАЯ\n\n\n\nI\n\nВсе счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.'

In [15]:
encoded[:110]

array([ 99,  77,  93,  94, 102,   1,  91,  82,  92,  79,  77, 105,   0,
         0,   0,   0,  30,   0,   0,  79, 123, 111,   1, 123, 129, 106,
       123, 124, 117, 114, 108, 133, 111,   1, 123, 111, 118, 134, 114,
         1, 121, 120, 127, 120, 112, 114,   1, 110, 122, 125, 109,   1,
       119, 106,   1, 110, 122, 125, 109, 106,   7,   1, 116, 106, 112,
       110, 106, 137,   1, 119, 111, 123, 129, 106, 123, 124, 117, 114,
       108, 106, 137,   1, 123, 111, 118, 134, 137,   1, 119, 111, 123,
       129, 106, 123, 124, 117, 114, 108, 106,   1, 121, 120,   8, 123,
       108, 120, 111, 118, 125,   9], dtype=int32)

In [16]:
len(vocab)

140

In [17]:
def get_batches(arr, n_seqs, n_steps):
    '''Создаем генератор, который возвращает пакеты размером
       n_seqs x n_steps из массива arr.
       
       Аргументы
       ---------
       arr: Массив, из которого получаем пакеты
       n_seqs: Batch size, количество последовательностей в пакете
       n_steps: Sequence length, сколько "шагов" делаем в пакете
    '''
    # Считаем количество символов на пакет и количество пакетов, которое можем сформировать
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Сохраняем в массиве только символы, которые позволяют сформировать целое число пакетов
    arr = arr[:n_batches * characters_per_batch]
    
    # Делаем reshape 1D -> 2D, используя n_seqs как число строк, как на картинке
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # пакет данных, который будет подаваться на вход сети
        x = arr[:, n:n+n_steps]
        # целевой пакет, с которым будем сравнивать предсказание, получаем сдвиганием "x" на один символ вперед
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [18]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:5, :5])
print('\ny\n', y[:5, :5])

x
 [[ 99  77  93  94 102]
 [  1 110 108 114 112]
 [ 79 120 124   1 120]
 [114 119   1 109 120]
 [123 121 122 106 108]]

y
 [[ 77  93  94 102   1]
 [110 108 114 112 111]
 [120 124   1 120 124]
 [119   1 109 120 108]
 [121 122 106 108 111]]


In [19]:
def build_inputs(batch_size, num_steps):
    ''' Определяем placeholder'ы для входных, целевых данных, а также вероятности drop out
    
        Аргументы
        ---------
        batch_size: Batch size, количество последовательностей в пакете
        num_steps: Sequence length, сколько "шагов" делаем в пакете
        
    '''
    # Объявляем placeholder'ы
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Placeholder для вероятности drop out
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [20]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Строим LSTM ячейку.
    
        Аргументы
        ---------
        keep_prob: Скаляр (tf.placeholder) для dropout keep probability
        lstm_size: Размер скрытых слоев в LSTM ячейках
        num_layers: Количество LSTM слоев
        batch_size: Batch size

    '''
    ### Строим LSTM ячейку
    
    def build_cell(lstm_size, keep_prob):
        # Начинаем с базовой LSTM ячейки
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Добавляем dropout к ячейке
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Стэкируем несколько LSTM слоев для придания глубины нашему deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    # Инициализируем начальное состояние LTSM ячейки
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [21]:
def build_output(lstm_output, in_size, out_size):
    ''' Строим softmax слой и возвращаем результат его работы.
    
        Аргументы
        ---------
        
        x: Входящий от LSTM тензор
        in_size: Размер входящего тензора, (кол-во LSTM юнитов скрытого слоя)
        out_size: Размер softmax слоя (объем словаря)
    
    '''

    # вытягиваем и решэйпим тензор, выполняя преобразование 3D -> 2D
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Соединяем результат LTSM слоев с softmax слоем
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Считаем logit-функцию
    logits = tf.matmul(x, softmax_w) + softmax_b
    # Используем функцию softmax для получения предсказания
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [22]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Считаем функцию потери на основании значений logit-функции и целевых значений.
    
        Аргументы
        ---------
        logits: значение logit-функции
        targets: целевые значения, с которыми сравниваем предсказания
        lstm_size: Количество юнитов в LSTM слое
        num_classes: Количество классов в целевых значениях (размер словаря)
        
    '''
    # Делаем one-hot кодирование целевых значений и решейпим по образу и подобию logits
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Считаем значение функции потери softmax cross entropy loss и возвращаем среднее значение
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [23]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Строим оптимизатор для обучения, используя обрезку градиента.
    
        Arguments:
        loss: значение функции потери
        learning_rate: параметр скорости обучения
    
    '''
    
    # Оптимизатор для обучения, обрезка градиента для контроля "взрывающихся" градиентов
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [24]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # Мы будем использовать эту же сеть для сэмплирования (генерации текста),
        # при этом будем подавать по одному символу за один раз
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Получаем input placeholder'ы
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Строим LSTM ячейку
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Прогоняем данные через RNN слои
        # Делаем one-hot кодирование входящих данных
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Прогоняем данные через RNN и собираем результаты
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Получаем предсказания (softmax) и результат logit-функции
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Считаем потери и оптимизируем (с обрезкой градиента)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [25]:
batch_size = 100        # Размер пакета
num_steps = 100         # Шагов в пакете
lstm_size = 512         # Количество LSTM юнитов в скрытом слое
num_layers = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob = 0.5         # Dropout keep probability

In [ ]:
epochs = 30
# Сохраняться каждый N итераций
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Можно раскомментировать строчку ниже и продолжить обучение с checkpoint'а
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Обучаем сеть
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/30...  Training Step: 1...  Training loss: 4.9429...  0.3129 sec/batch
Epoch: 1/30...  Training Step: 2...  Training loss: 4.8686...  0.3276 sec/batch
Epoch: 1/30...  Training Step: 3...  Training loss: 4.4622...  0.2294 sec/batch
Epoch: 1/30...  Training Step: 4...  Training loss: 5.0477...  0.2836 sec/batch
Epoch: 1/30...  Training Step: 5...  Training loss: 4.1799...  0.2826 sec/batch
Epoch: 1/30...  Training Step: 6...  Training loss: 3.9392...  0.1968 sec/batch
Epoch: 1/30...  Training Step: 7...  Training loss: 3.9166...  0.1785 sec/batch
Epoch: 1/30...  Training Step: 8...  Training loss: 3.7942...  0.2283 sec/batch
Epoch: 1/30...  Training Step: 9...  Training loss: 3.7031...  0.1859 sec/batch
Epoch: 1/30...  Training Step: 10...  Training loss: 3.6427...  0.1794 sec/batch
Epoch: 1/30...  Training Step: 11...  Training loss: 3.6076...  0.2558 sec/batch
Epoch: 1/30...  Training Step: 12...  Training loss: 3.5756...  0.2708 sec/batch
Epoch: 1/30...  Training Step: 13... 

Epoch: 1/30...  Training Step: 103...  Training loss: 3.8288...  0.2437 sec/batch
Epoch: 1/30...  Training Step: 104...  Training loss: 3.7700...  0.1580 sec/batch
Epoch: 1/30...  Training Step: 105...  Training loss: 3.6463...  0.2126 sec/batch
Epoch: 1/30...  Training Step: 106...  Training loss: 3.5880...  0.1955 sec/batch
Epoch: 1/30...  Training Step: 107...  Training loss: 3.4745...  0.1558 sec/batch
Epoch: 1/30...  Training Step: 108...  Training loss: 3.3973...  0.2311 sec/batch
Epoch: 1/30...  Training Step: 109...  Training loss: 3.2636...  0.1685 sec/batch
Epoch: 1/30...  Training Step: 110...  Training loss: 3.2536...  0.2097 sec/batch
Epoch: 1/30...  Training Step: 111...  Training loss: 3.2231...  0.1792 sec/batch
Epoch: 1/30...  Training Step: 112...  Training loss: 3.2347...  0.2429 sec/batch
Epoch: 1/30...  Training Step: 113...  Training loss: 3.2414...  0.2803 sec/batch
Epoch: 1/30...  Training Step: 114...  Training loss: 3.2465...  0.2693 sec/batch
Epoch: 1/30...  

Epoch: 2/30...  Training Step: 203...  Training loss: 2.7434...  0.2690 sec/batch
Epoch: 2/30...  Training Step: 204...  Training loss: 2.7220...  0.1998 sec/batch
Epoch: 2/30...  Training Step: 205...  Training loss: 2.7286...  0.2150 sec/batch
Epoch: 2/30...  Training Step: 206...  Training loss: 2.7461...  0.2697 sec/batch
Epoch: 2/30...  Training Step: 207...  Training loss: 2.7331...  0.2774 sec/batch
Epoch: 2/30...  Training Step: 208...  Training loss: 2.7159...  0.2515 sec/batch
Epoch: 2/30...  Training Step: 209...  Training loss: 2.7216...  0.3241 sec/batch
Epoch: 2/30...  Training Step: 210...  Training loss: 2.6989...  0.2039 sec/batch
Epoch: 2/30...  Training Step: 211...  Training loss: 2.6953...  0.1740 sec/batch
Epoch: 2/30...  Training Step: 212...  Training loss: 2.6995...  0.2502 sec/batch
Epoch: 2/30...  Training Step: 213...  Training loss: 2.6720...  0.1954 sec/batch
Epoch: 2/30...  Training Step: 214...  Training loss: 2.6732...  0.2288 sec/batch
Epoch: 2/30...  

Epoch: 2/30...  Training Step: 303...  Training loss: 2.4387...  0.2277 sec/batch
Epoch: 2/30...  Training Step: 304...  Training loss: 2.4836...  0.2720 sec/batch
Epoch: 2/30...  Training Step: 305...  Training loss: 2.4754...  0.2317 sec/batch
Epoch: 2/30...  Training Step: 306...  Training loss: 2.4751...  0.2181 sec/batch
Epoch: 2/30...  Training Step: 307...  Training loss: 2.4387...  0.1569 sec/batch
Epoch: 2/30...  Training Step: 308...  Training loss: 2.4396...  0.2585 sec/batch
Epoch: 2/30...  Training Step: 309...  Training loss: 2.4463...  0.2909 sec/batch
Epoch: 2/30...  Training Step: 310...  Training loss: 2.4377...  0.1860 sec/batch
Epoch: 2/30...  Training Step: 311...  Training loss: 2.4330...  0.2384 sec/batch
Epoch: 2/30...  Training Step: 312...  Training loss: 2.4429...  0.2065 sec/batch
Epoch: 2/30...  Training Step: 313...  Training loss: 2.4358...  0.1637 sec/batch
Epoch: 2/30...  Training Step: 314...  Training loss: 2.4515...  0.1892 sec/batch
Epoch: 2/30...  

Epoch: 3/30...  Training Step: 403...  Training loss: 2.2942...  0.1646 sec/batch
Epoch: 3/30...  Training Step: 404...  Training loss: 2.2840...  0.2340 sec/batch
Epoch: 3/30...  Training Step: 405...  Training loss: 2.3145...  0.2608 sec/batch
Epoch: 3/30...  Training Step: 406...  Training loss: 2.2991...  0.2679 sec/batch
Epoch: 3/30...  Training Step: 407...  Training loss: 2.3105...  0.1857 sec/batch
Epoch: 3/30...  Training Step: 408...  Training loss: 2.2730...  0.1899 sec/batch
Epoch: 3/30...  Training Step: 409...  Training loss: 2.3078...  0.2445 sec/batch
Epoch: 3/30...  Training Step: 410...  Training loss: 2.2686...  0.2447 sec/batch
Epoch: 3/30...  Training Step: 411...  Training loss: 2.2840...  0.1581 sec/batch
Epoch: 3/30...  Training Step: 412...  Training loss: 2.2755...  0.1713 sec/batch
Epoch: 3/30...  Training Step: 413...  Training loss: 2.2984...  0.1938 sec/batch
Epoch: 3/30...  Training Step: 414...  Training loss: 2.3180...  0.2541 sec/batch
Epoch: 3/30...  

...

Epoch: 16/30...  Training Step: 2578...  Training loss: 1.4805...  0.2208 sec/batch
Epoch: 16/30...  Training Step: 2579...  Training loss: 1.4549...  0.1809 sec/batch
Epoch: 16/30...  Training Step: 2580...  Training loss: 1.4301...  0.2078 sec/batch
Epoch: 16/30...  Training Step: 2581...  Training loss: 1.4569...  0.2071 sec/batch
Epoch: 16/30...  Training Step: 2582...  Training loss: 1.4442...  0.2007 sec/batch
Epoch: 16/30...  Training Step: 2583...  Training loss: 1.4598...  0.2233 sec/batch
Epoch: 16/30...  Training Step: 2584...  Training loss: 1.4386...  0.1767 sec/batch
Epoch: 16/30...  Training Step: 2585...  Training loss: 1.4437...  0.2486 sec/batch
Epoch: 16/30...  Training Step: 2586...  Training loss: 1.4564...  0.2319 sec/batch
Epoch: 16/30...  Training Step: 2587...  Training loss: 1.4955...  0.2415 sec/batch
Epoch: 16/30...  Training Step: 2588...  Training loss: 1.4811...  0.2508 sec/batch
Epoch: 16/30...  Training Step: 2589...  Training loss: 1.4550...  0.2090 se

Epoch: 16/30...  Training Step: 2676...  Training loss: 1.4425...  0.2015 sec/batch
Epoch: 16/30...  Training Step: 2677...  Training loss: 1.4571...  0.2012 sec/batch
Epoch: 16/30...  Training Step: 2678...  Training loss: 1.4602...  0.1883 sec/batch
Epoch: 16/30...  Training Step: 2679...  Training loss: 1.4649...  0.1662 sec/batch
Epoch: 16/30...  Training Step: 2680...  Training loss: 1.4889...  0.2281 sec/batch
Epoch: 16/30...  Training Step: 2681...  Training loss: 1.4970...  0.2684 sec/batch
Epoch: 16/30...  Training Step: 2682...  Training loss: 1.4249...  0.2506 sec/batch
Epoch: 16/30...  Training Step: 2683...  Training loss: 1.4302...  0.2199 sec/batch
Epoch: 16/30...  Training Step: 2684...  Training loss: 1.4837...  0.2130 sec/batch
Epoch: 16/30...  Training Step: 2685...  Training loss: 1.4511...  0.1697 sec/batch
Epoch: 16/30...  Training Step: 2686...  Training loss: 1.4879...  0.1657 sec/batch
Epoch: 16/30...  Training Step: 2687...  Training loss: 1.4304...  0.2276 se

Epoch: 17/30...  Training Step: 2774...  Training loss: 1.4278...  0.2417 sec/batch
Epoch: 17/30...  Training Step: 2775...  Training loss: 1.4514...  0.2593 sec/batch
Epoch: 17/30...  Training Step: 2776...  Training loss: 1.4654...  0.2266 sec/batch
Epoch: 17/30...  Training Step: 2777...  Training loss: 1.4515...  0.2270 sec/batch
Epoch: 17/30...  Training Step: 2778...  Training loss: 1.4179...  0.2022 sec/batch
Epoch: 17/30...  Training Step: 2779...  Training loss: 1.4642...  0.2007 sec/batch
Epoch: 17/30...  Training Step: 2780...  Training loss: 1.4579...  0.2643 sec/batch
Epoch: 17/30...  Training Step: 2781...  Training loss: 1.4464...  0.1935 sec/batch
Epoch: 17/30...  Training Step: 2782...  Training loss: 1.4319...  0.2011 sec/batch
Epoch: 17/30...  Training Step: 2783...  Training loss: 1.4266...  0.2335 sec/batch
Epoch: 17/30...  Training Step: 2784...  Training loss: 1.4428...  0.2175 sec/batch
Epoch: 17/30...  Training Step: 2785...  Training loss: 1.4614...  0.1846 se

Epoch: 17/30...  Training Step: 2873...  Training loss: 1.4346...  0.2119 sec/batch
Epoch: 17/30...  Training Step: 2874...  Training loss: 1.4156...  0.1789 sec/batch
Epoch: 17/30...  Training Step: 2875...  Training loss: 1.4138...  0.2171 sec/batch
Epoch: 17/30...  Training Step: 2876...  Training loss: 1.3976...  0.2849 sec/batch
Epoch: 17/30...  Training Step: 2877...  Training loss: 1.4239...  0.1885 sec/batch
Epoch: 17/30...  Training Step: 2878...  Training loss: 1.4357...  0.2147 sec/batch
Epoch: 17/30...  Training Step: 2879...  Training loss: 1.4149...  0.1570 sec/batch
Epoch: 17/30...  Training Step: 2880...  Training loss: 1.4522...  0.2262 sec/batch
Epoch: 17/30...  Training Step: 2881...  Training loss: 1.4230...  0.2540 sec/batch
Epoch: 17/30...  Training Step: 2882...  Training loss: 1.4554...  0.2389 sec/batch
Epoch: 17/30...  Training Step: 2883...  Training loss: 1.4318...  0.1762 sec/batch
Epoch: 17/30...  Training Step: 2884...  Training loss: 1.4283...  0.1608 se

Epoch: 18/30...  Training Step: 2971...  Training loss: 1.4451...  0.2435 sec/batch
Epoch: 18/30...  Training Step: 2972...  Training loss: 1.4326...  0.1682 sec/batch
Epoch: 18/30...  Training Step: 2973...  Training loss: 1.4416...  0.1586 sec/batch
Epoch: 18/30...  Training Step: 2974...  Training loss: 1.4038...  0.1959 sec/batch
Epoch: 18/30...  Training Step: 2975...  Training loss: 1.4357...  0.2058 sec/batch
Epoch: 18/30...  Training Step: 2976...  Training loss: 1.4139...  0.1718 sec/batch
Epoch: 18/30...  Training Step: 2977...  Training loss: 1.4355...  0.2481 sec/batch
Epoch: 18/30...  Training Step: 2978...  Training loss: 1.4507...  0.2183 sec/batch
Epoch: 18/30...  Training Step: 2979...  Training loss: 1.4484...  0.2244 sec/batch
Epoch: 18/30...  Training Step: 2980...  Training loss: 1.4039...  0.1541 sec/batch
Epoch: 18/30...  Training Step: 2981...  Training loss: 1.4120...  0.2127 sec/batch
Epoch: 18/30...  Training Step: 2982...  Training loss: 1.4064...  0.1799 se

In [27]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"

In [28]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [29]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="Гостиная Анны Павловны начала понемногу наполняться."):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [30]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1000_l512.ckpt
Гостиная Анны Павловны начала понемногу наполняться.

Семей придужалось в песелица и своех серделовости с теперь не выславится и высольным не выпоснимаясь, и она была одной совершенно вы сказала ничего она сказал на совериенностью в доменной слишкого слиданным полежалим, от намило и пать и ответа и с как было понимал, он не все помочула и, привав им в постерной. Она вишал настольно престили его продотные волости, к тому которые он странное поледу, что предстание оточно столо вставался на том, что она ничестна не мыгатеть. Она все стала оставал вся себя верово не межно и непрастние свои поднавшие стало. Он получал совели встому, что он не случно непись на подошло его не могла. Она было слешить ни слывали, которое приведить ей, на теперь своима и придалась из к него с своим с нимименния, подомнила она сона понавалась своюго стерным, кордаты, как начто беселие возно все друго и в думой весловие все все весело, ни он при